### Выполнила: Ковалева Екатерина Сергеевна  НИЯУ МИФИ группа М24-525

## Import

In [2]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import pandas as pd
import requests
import time
import numpy as np
import re
import seaborn as sns


## Создание расширенных признаков

**Здесь возьмем наш сохраненный, с заполнением пропусков OpenLibrary датасет и продолжим работать с ним для генерации дополнительных признаков. Остальные датасеты остались без изменений.**

In [27]:
books = pd.read_csv('/home/administrator/Desktop/IDE/Recomendation/books_new.csv')
ratings = pd.read_csv('/home/administrator/Desktop/IDE/Recomendation/ratings.csv')
tags =  pd.read_csv('/home/administrator/Desktop/IDE/Recomendation/tags.csv')
book_tags = pd.read_csv('/home/administrator/Desktop/IDE/Recomendation/book_tags.csv')


In [6]:
books.head(3)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


## 1.1Признаки пользователей: средний рейтинг, количество оценок, активность.


Создадим признаки, описывающие поведение пользователей:

- **`avg_user_rating`** — средняя оценка, выставленная пользователем. Помогает понять, является ли пользователь «строгим» или «щедрым» рецензентом.  
- **`user_ratings_count`** — общее количество оценок, выставленных пользователем. Характеризует активность и вовлечённость.

In [28]:
# Группируем данные по user_id и вычисляем количество и среднее значение оценок
user_features = ratings.groupby('user_id')['rating'].agg(['count', 'mean']).reset_index()

user_features.columns = ['user_id', 'user_ratings_count', 'user_avg_rating']


In [16]:
user_features

,user_id,user_ratings_count,user_avg_rating
0,1,117,3.589744
1,2,65,4.415385
2,3,91,1.736264
3,4,134,3.768657
4,5,100,4.040000
...,...,...,...
53419,53420,110,3.345455
53420,53421,110,4.609091
53421,53422,130,4.215385
53422,53423,77,4.454545


## 1.2 Признаки книг: популярность, разнообразие оценок, тематические категории

Cоздадим признаки, описывающие каждую книгу.

weighted_rating: Взвешенный рейтинг, который учитывает не только среднюю оценку, но и количество голосов.

rating_std_dev: Стандартное отклонение оценок для книги (разброс мнений).

tags_list: Список тематических тегов для каждой книги.

In [29]:
C = books['average_rating'].mean()              # Популярность (Взвешенный рейтинг)
m = books['ratings_count'].quantile(0.75)

books['weighted_rating'] = (books['ratings_count'] / (books['ratings_count'] + m)) * books['average_rating'] + \
                         (m / (books['ratings_count'] + m)) * C

In [30]:
rating_std = ratings.groupby('book_id')['rating'].std().reset_index()  ## Разнообразие оценок (Стандартное отклонение)
rating_std.rename(columns={'rating': 'rating_std_dev'}, inplace=True)
books = pd.merge(books, rating_std, on='book_id', how='left')
books['rating_std_dev'].fillna(0, inplace=True)

/tmp/ipykernel_1999624/1821426885.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  books['rating_std_dev'].fillna(0, inplace=True)


In [31]:
book_tags_merged = pd.merge(book_tags, tags, on='tag_id')   ##  Тематические категории (Теги)
book_tags_agg = book_tags_merged.groupby('goodreads_book_id')['tag_name'].apply(list).reset_index()
book_tags_agg.rename(columns={'tag_name': 'tags_list'}, inplace=True)
books = pd.merge(books, book_tags_agg, on='goodreads_book_id', how='left')

## 1.3 Создание признаков взаимодействий

In [ ]:
# Сначала объединяем все данные в один датафрейм
data = pd.merge(ratings, user_features, on='user_id', how='left')
data = pd.merge(data, books, on='book_id', how='left')

# Теперь создаем признаки взаимодействия
data['rating_deviation_user'] = data['rating'] - data['user_avg_rating']
data['rating_deviation_book'] = data['rating'] - data['average_rating']


print("Финальный объединенный датасет с признаками взаимодействия .")
print("Количество строк:", len(data))
print("Количество столбцов:", len(data.columns))


Финальный объединенный датасет с признаками взаимодействия готов.
Количество строк: 5976479
Количество столбцов: 32


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 32 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   user_id                    int64  
 1   book_id                    int64  
 2   rating                     int64  
 3   user_ratings_count         int64  
 4   user_avg_rating            float64
 5   goodreads_book_id          int64  
 6   best_book_id               int64  
 7   work_id                    int64  
 8   books_count                int64  
 9   isbn                       object 
 10  isbn13                     float64
 11  authors                    object 
 12  original_publication_year  float64
 13  original_title             object 
 14  title                      object 
 15  language_code              object 
 16  average_rating             float64
 17  ratings_count              int64  
 18  work_ratings_count         int64  
 19  work_text_reviews_count    int64  
 20  ra

In [35]:
data.to_parquet('data_advanced.parquet', index=False)